In [1]:
import folium
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon
import pandas as pd
import numpy as np

In [2]:
df_state = pd.read_csv('states.csv')

In [3]:
df_state.head()

,state,latitude,longitude,name
0,AK,63.588753,-154.493062,Alaska
1,AL,32.318231,-86.902298,Alabama
2,AR,35.201050,-91.831833,Arkansas
3,AZ,34.048928,-111.093731,Arizona
4,CA,36.778261,-119.417932,California


In [4]:
df_data_analyst = pd.read_csv('data_analyst.csv')

In [5]:
df_data_analyst = df_data_analyst[['job_location', 'company', 'search_country']]

In [6]:
df_location_US = df_data_analyst[df_data_analyst['search_country'] == 'United States']

In [7]:
df_location_US.head()

,job_location,company,search_country
0,"Durham, NC",Duke University,United States
1,"Dublin, CA",ClickJobs.io,United States
2,"Hagerstown, MD",JLG Industries,United States
3,"Newark, DE",The Judge Group,United States
4,"Newark, DE",Dice,United States


In [8]:
df_location_US.isna().sum()

job_location      2
company           0
search_country    0
dtype: int64

In [9]:
df_location_US.dropna(inplace=True)

C:\Users\vanhu\AppData\Local\Temp\ipykernel_27168\4289844609.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location_US.dropna(inplace=True)


In [10]:
def extract_State(s):
    return ''.join([c for c in s[-2:] if c.isupper()])

In [11]:
df_location_US['state'] = df_location_US['job_location'].apply(extract_State)

C:\Users\vanhu\AppData\Local\Temp\ipykernel_27168\1320856033.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_location_US['state'] = df_location_US['job_location'].apply(extract_State)


In [12]:
df_location_US.head()

,job_location,company,search_country,state
0,"Durham, NC",Duke University,United States,NC
1,"Dublin, CA",ClickJobs.io,United States,CA
2,"Hagerstown, MD",JLG Industries,United States,MD
3,"Newark, DE",The Judge Group,United States,DE
4,"Newark, DE",Dice,United States,DE


In [13]:
df_location_US = pd.merge(df_state, df_location_US, how = 'inner', on = 'state')

In [14]:
df_location_US.drop(columns='name',inplace=True)

In [15]:
#df_location_US.to_csv('location_US.csv', index=False)

In [16]:
df_location_US = pd.read_csv('location_US.csv')

In [17]:
df_location_US.head()

,state,latitude,longitude,job_location,company,search_country
0,AK,63.588753,-154.493062,"Anchorage, AK","IRS, Office of Chief Counsel",United States
1,AK,63.588753,-154.493062,"Fairbanks, AK",Foundation Health Partners,United States
2,AK,63.588753,-154.493062,"Anchorage, AK",Arctic Slope Regional Corporation,United States
3,AK,63.588753,-154.493062,"Fairbanks, AK",Foundation Health Partners,United States
4,AK,63.588753,-154.493062,"Anchorage, AK",GCI Communication Corp.,United States


In [18]:
df_state = df_location_US.drop_duplicates(subset='state')

In [19]:
df_state.shape

(51, 6)

In [20]:
site_map = folium.Map(location=[37.0902, -95.7129], zoom_start=5)

In [21]:
for i in range(len(df_state)):
    circle = folium.Circle(df_state[['latitude', 'longitude']].values[i], radius=1000, color='#d35400', fill=True).add_child(folium.Popup(f'{df_state['state'].values[i]}'))
    marker = folium.map.Marker(
        df_state[['latitude', 'longitude']].values[i],
        icon=DivIcon(
            icon_size=(10,10),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % f'{df_state['state'].values[i]}',
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map

In [ ]:
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

for index, record in df_location_US.iterrows():
    marker = folium.Circle(
                            location=[record['latitude'], record['longitude']] ,
                            tiles=f'{record['state']}',
                                                                      
    )
    marker_cluster.add_child(marker)
site_map